Постановка задачи

В этой же папке на диске лежит файл dataset.csv

Ваша задача провести анализ данных, построить и оценить модель классификации. Выбор метрик и моделей за вами. Решение должно быть загружено в формате .ipynb или .html

Оценка работ будет производиться по следующим критериям: 

- Качественный EDA
- Корректные и обоснованные трансформации данных
- Корректное и обоснованное обучение и оценка модели
- Качество кода и чистота jupyter notebook


Описание данных

Это набор данных HR. Задача состоит в том, чтобы проверить, получил ли сотрудник повышение или нет.
Названия колонок:
	•	employee_id
	•	department
	•	region
	•	education
	•	gender
	•	recruitment_channel
	•	nooftrainings
	•	age
	•	previousyearrating
	•	lengthofservice
	•	KPIs_met >80%
	•	awards_won?
	•	avgtrainingscore
	•	is_promoted

Если по названию колонки все еще не понятно, что в ней содержится, мы ожидаем, что вы сделаете свои предположения на основе анализа данных


In [ ]:
import warnings
import os
import os.path as osp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
warnings.filterwarnings("ignore")


In [ ]:
df = pd.read_csv("dataset.csv")
df

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
missing_values = df.isnull().sum()

data_types = df.dtypes

In [ ]:
miss_v_df = pd.DataFrame(zip(*[missing_values.index, missing_values.values, data_types.values]),
                         columns=["columns_name", "missing_values", "data_types"])
miss_v_df

`is_promoted` <- Наш таргет.

In [ ]:
all_columns = ["department", "region", "education", "gender", "recruitment_channel", "no_of_trainings", "age",
               "previous_year_rating", "length_of_service", "KPIs_met >80%", "awards_won?", "avg_training_score",
               "is_promoted"]

In [ ]:
figsize = (14, 18)
plt.figure(figsize=figsize)

for i, column in enumerate(all_columns, 1):
    plt.subplot(5, 3, i)
    sns.countplot(data=df, x=column, palette='Set2')
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=figsize)

for i, column in enumerate(all_columns, 1):
    plt.subplot(5, 3, i)
    if df[column].dtype == 'object':
        sns.countplot(data=df, x=column, palette='Set2')
        plt.xticks(rotation=90)
    else:
        sns.histplot(df[column], kde=True, palette='Set2')
    plt.title(f'Distribution of {column}')

plt.tight_layout()
plt.show()


In [ ]:
numerical_features = ['age', 'length_of_service', 'avg_training_score']
categorical_features = ['awards_won?', 'KPIs_met >80%', 'department', 'region', 'education', 'gender',
                        'previous_year_rating', 'no_of_trainings', 'recruitment_channel']

Давайте посмотрим чем можно заполнить пропуски

In [ ]:
education_counts = df['education'].value_counts()
education_mode = education_counts.idxmax()
education_mode

In [ ]:
previous_year_rating_mean = df['previous_year_rating'].mean()
previous_year_rating_median = df['previous_year_rating'].median()
previous_year_rating_mode = df['previous_year_rating'].mode()[0]

plt.figure(figsize=(25, 7))
plt.subplot(1, 2, 2)
sns.histplot(df['previous_year_rating'], kde=True, palette='Set2')
plt.axvline(previous_year_rating_mean, color='g', linestyle='--', label=f'Mean: {previous_year_rating_mean:.2f}')
plt.axvline(previous_year_rating_median, color='b', linestyle='--', label=f'Median: {previous_year_rating_median}')
plt.axvline(previous_year_rating_mode, color='r', linestyle='--', label=f'Mode: {previous_year_rating_mode}')
plt.title('Distribution of Previous Year Rating')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
df['education'].fillna('Bachelor\'s', inplace=True)
df['previous_year_rating'].fillna(3.0, inplace=True)

In [ ]:
missing_values = df.isnull().sum()

data_types = df.dtypes

miss_v_df = pd.DataFrame(zip(*[missing_values.index, missing_values.values, data_types.values]),
                         columns=["columns_name", "missing_values", "data_types"])
miss_v_df

In [ ]:
previous_year_rating_mean = df['previous_year_rating'].mean()
previous_year_rating_median = df['previous_year_rating'].median()
previous_year_rating_mode = df['previous_year_rating'].mode()[0]

plt.figure(figsize=(25, 7))
plt.subplot(1, 2, 2)
sns.histplot(df['previous_year_rating'], kde=True, palette='Set2')
plt.axvline(previous_year_rating_mean, color='g', linestyle='--', label=f'Mean: {previous_year_rating_mean:.2f}')
plt.axvline(previous_year_rating_median, color='b', linestyle='--', label=f'Median: {previous_year_rating_median}')
plt.axvline(previous_year_rating_mode, color='r', linestyle='--', label=f'Mode: {previous_year_rating_mode}')
plt.title('Distribution of Previous Year Rating')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
correlation_matrix = df[numerical_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numerical Features')
plt.show()

Существует умеренная положительная корреляция (0,66) между возрастом и продолжительностью трудового стажа, что ожидаемо, поскольку пожилые сотрудники, как правило, имеют более длительный стаж работы

Показатель `avg_training_score` показывает очень слабую корреляцию как с возрастом (-0,048), так и с продолжительностью трудового стажа (-0,038), что указывает на то, что показатели обучения относительно независимы от возраста или стажа сотрудника.

In [ ]:
plt.figure(figsize=(15, 8))


def plot_promote_rate(df, column):
    sns.countplot(data=df, x=column, hue='is_promoted', palette='Set2')
    plt.title(f'Relationship between {column} and Promotion')
    plt.xticks(rotation=90)

    # Calculate and add ratios
    uniq_values_of_column = df[column].unique()
    for uniq_value_of_column in uniq_values_of_column:
        count_promoted = df[(df[column] == uniq_value_of_column) & (df["is_promoted"] == 1)].shape[0]
        count_not_promoted = df[(df[column] == uniq_value_of_column) & (df["is_promoted"] == 0)].shape[0]
        ratio = count_promoted / (count_not_promoted + count_promoted)
        plt.text(uniq_value_of_column, count_promoted, f'{ratio:.2f}', ha='center', va='bottom')


plot_promote_rate(df, column="region")
plt.tight_layout()
plt.show()

plt.figure(figsize=(22, 18))

categorical_features_except_region = [c for c in categorical_features if c != "region"]

for i, column in enumerate(categorical_features_except_region, 1):
    plt.subplot(4, 2, i)
    plot_promote_rate(df, column)

plt.tight_layout()
plt.show()


### Анализ взаимосвязей между категориальными признаками и статусом повышения

1. **Награды**:
   - Сотрудники, получившие награды, имеют значительно более высокий процент повышения (44%) по сравнению с теми, кто не получал награды (8%).

2. **KPIs выполнены >80%**:
   - Сотрудники, выполнившие более 80% своих KPIs, имеют более высокий процент повышения (17%) по сравнению с теми, кто этого не сделал (4%).

3. **Отдел**:
   - Наибольший процент повышений наблюдается в отделах `Технологии` (11%) и `Аналитика` (10%), в то время как в других отделах процент повышений колеблется от 5% до 10%.

4. **Регион**:
   - Повышения распределены по всем регионам, с процентом повышения, варьирующимся от 2% до 14%. Наибольший процент повышений наблюдается в регионах `region_4` (14%) и `region_1` (12%).

5. **Образование**:
   - Сотрудники с образованием уровня `Магистр и выше` имеют наибольший процент повышения (10%) по сравнению с `Бакалавром` (8%) и `Ниже среднего` (8%).

6. **Пол**:
   - Мужчины и женщины имеют схожие показатели повышения (8% и 9% соответственно).

7. **Рейтинг за предыдущий год**:
   - Высокие рейтинги за предыдущий год (4 и 5) ассоциируются с более высоким процентом повышений (7% и 16% соответственно) по сравнению с низкими рейтингами (1% для рейтинга 2 и 4% для рейтинга 3).

8. **Количество тренировок**:
   - Сотрудники, прошедшие больше тренировок (2-3), имеют более высокий процент повышения (8% и 7% соответственно) по сравнению с теми, у кого была только одна тренировка (8%).

9. **Канал набора**:
   - Канал `sourcing` имеет более высокий процент повышений (9%) по сравнению с `other` (8%) и `referred` (12%).
